In [1]:
import openai
import nest_asyncio
nest_asyncio.apply()

In [2]:
openai.api_key = 'YOUR_API_KEY'

## Setting Up the Query Tools

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools('metagpt.pdf', 'metagpt')

## setup Function Calling Agent

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo', temperature=0)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

agent = AgentRunner(agent_worker)

In [6]:
# agent.query() --> for single-turn queries i.e. doesn't reatain context from previous interactions
response = agent.query(
    "Tell me about the agent roles in MetaGPT,"
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT,and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineers, and QA Engineer. Each role has specific responsibilities such as generating the Product Requirement Document and competitive analysis, designing system architecture and technical specifications, breaking down tasks for Engineers, developing code based on specifications, and ensuring software quality through unit testing. These roles collaborate to efficiently handle different aspects of software development tasks within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each oth

In [7]:
# for multi-turn conversations; remembers context throughout the session
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, while MBPP comprises 427 Python tasks covering core concepts and standard library features. The SoftwareDev dataset includes 70 representative examples of software development tasks with diverse scopes, such as mini-games, image processing algorithms, and data visualization. These datasets were utilized to evaluate the performance of MetaGPT in code generation tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval consists of 164 handwritten programming tasks, MBPP comprises 427 Python tasks, and the SoftwareDev datase

## Lower-Level Debuggability and Control

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT,"
    "and then how they communicate with each other."
)

In [10]:
# Executes a specific step of the given task using its task_id, 
# typically in a multi-step workflow or background process.
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT,and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT are the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating the Product Requirement Document, designing system architecture, breaking down projects into tasks, coding based on defined tasks, creating unit tests, and ensuring software quality. These roles collaborate to facilitate the development process effectively within MetaGPT.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, a publish-subscrib

In [13]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id} : {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 2d650c08-592d-4436-8970-62d0730d73f8 : 1
The agent roles in MetaGPT are the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating the Product Requirement Document, designing system architecture, breaking down projects into tasks, coding based on defined tasks, creating unit tests, and ensuring software quality. These roles collaborate to facilitate the development process effectively within MetaGPT.


In [14]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id} : {len(upcoming_steps)}")

Num upcoming steps for task 2d650c08-592d-4436-8970-62d0730d73f8 : 1


In [15]:
upcoming_steps[0]

TaskStep(task_id='2d650c08-592d-4436-8970-62d0730d73f8', step_id='4b15b89a-c94c-4e21-9bb4-0bc48e1df8be', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [ ]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, the agent roles include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating the Product Requirement Document, designing system architecture, breaking down projects into tasks, coding based on defined tasks, creating unit tests, and ensuring software quality. These roles collaborate to facilitate the development process effectively within MetaGPT.

Agents in MetaGPT communicate with each other through structured communication interfaces, a publish-subscribe mechanism, a shared message pool, and a subscription mechanism. This allows them to exchange information efficiently, manage information overload by subscribing to role-specific interests, and ensure effective collaboration by filtering out irrelevant contexts.
True


In [17]:
response = agent.finalize_response(task.task_id)
print(str(response))

In MetaGPT, the agent roles include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities in the software development process, such as generating the Product Requirement Document, designing system architecture, breaking down projects into tasks, coding based on defined tasks, creating unit tests, and ensuring software quality. These roles collaborate to facilitate the development process effectively within MetaGPT.

Agents in MetaGPT communicate with each other through structured communication interfaces, a publish-subscribe mechanism, a shared message pool, and a subscription mechanism. This allows them to exchange information efficiently, manage information overload by subscribing to role-specific interests, and ensure effective collaboration by filtering out irrelevant contexts.
